In [47]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.memory import ConversationBufferMemory

In [60]:
# LLM 설정
llm = OpenAI(model = 'gpt-4o', temperature = 0.7)
# 대화의 맥락 유지를 위한 memory 설정
memory = ConversationBufferMemory(memory_key="history")

In [49]:
# 프롬프트 1: 회사/직책이 요구하는 것 추출
job_requirements_prompt = PromptTemplate(
    input_variables=['company', 'position'],
    template = (
        'Given the company name "{company}" and the position "{position}",'
        'identify the key skills, qualities, and responsibilities required for this role.'
    )
)
job_requirements_chain = LLMChain(llm=llm, prompt=job_requirements_prompt, memory=memory)

In [50]:
# 프롬프트 2: 요구되는 것을 바탕으로 면접 질문 생성
interview_questions_prompt = PromptTemplate(
    input_variables = ['requirements'],
    template = (
        'Based on the following job requirements: {requirements},'
        'generate three detailed and role-specific interview questions that evaluate these criteria.'
    )
)
interview_questions_chain = LLMChain(llm=llm, prompt=interview_questions_prompt, memory=memory)

In [51]:
# 프롬프트 3: 사용자 대답 평가
answer_evaluation_prompt = PromptTemplate(
    input_variables = ['question', 'user_answer'],
    template = (
        'Evaluate the following answer of the user to the question: {question}\n\n'
        "User's Answer: {user_answer}\n\n"
        'Provide an evaluation based on relevance, depth, and clarity.'
    )
)
answer_evaluation_chain = LLMChain(llm=llm, prompt=answer_evaluation_prompt, memory=memory)


In [52]:
# 프롬프트 4: 모범 답안 생성
model_answer_prompt = PromptTemplate(
    input_variables=['user_answer', 'evaluation'],
    template=(
        'Given the answer of user : {user_answer}, and the following evaluation: {evaluation},'
        'generate an improved and exemplary answer that reflects the best practices for interview responses.'
    )
)
model_answer_chain = LLMChain(llm=llm, prompt=model_answer_prompt, memory=memory)

In [58]:
# 통합 flow
def interview_simulation(company, position, user_answers):
    # 1. 회사/직책 요구사항 추출
    requirements = job_requirements_chain.run({"company": company, "position": position})
    print(f"Job Requirements:\n{requirements}\n")

    # 2. 면접 질문 생성
    questions = interview_questions_chain.run({"requirements": requirements})
    print(f"Generated Questions:\n{questions}\n")

    # 3. 사용자 답변 평가 및 모범 답안 생성
    for idx, (question, answer) in enumerate(zip(questions.split('\n'), user_answers), 1):
        evaluation = answer_evaluation_chain.run({"question": question, "user_answer": answer})
        print(f"Evaluation for Answer {idx}:\n{evaluation}\n")
        
        model_answer = model_answer_chain.run({"user_answer": answer, "evaluation": evaluation})
        print(f"Model Answer for Question {idx}:\n{model_answer}\n")

In [59]:
# 사용 예시
company_name = "OpenAI"
position_name = "개발 엔지니어"
user_responses = [
    "I have experience in building machine learning models and deploying them.",
    "I believe teamwork and communication are my strongest assets.",
    "I am passionate about learning and adapting to new technologies."
]
interview_simulation(company_name, position_name, user_responses)

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}